# **CCC-BIDMC Schedule Generator**

**Ayush Noori** <br>
**Crimson Care Collaborative Clinic** <br>
**May 19, 2024**

---

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Path management
from pathlib import Path

# Get downloads folder
downloads = Path.home() / "Downloads"

Read in the availability data.

In [2]:
# Read in form responses
form_path = "CCC-BIDMC Summer 2024 Returning Interest Form (Responses) - Form Responses 1.csv"
df = pd.read_csv(downloads / form_path)

# Set column names
df.columns = ['timestamp', 'first', 'last', 'role', 'return', 'epic', 'shifts', 'comments']
df["first"] = df["first"].str.strip()
df["last"] = df["last"].str.strip()

# Convert time stamp to datetime
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Take the most recent response
df = df.sort_values("timestamp").groupby(["first", "last"]).last().reset_index()
df = df[df["return"] == "Yes"]

Generate availability table.

In [3]:
# Generate summer clinic dates
start_date = "June 4, 2024"
end_date = "September 17, 2024"

# Weekly dates on Tuesdays
dates = pd.date_range(start=start_date, end=end_date, freq="W-TUE")
dates = dates.strftime("%-m/%-d/%Y").values

# Create a dataframe with name, role, and dates
availability = pd.DataFrame(columns=["Name", "Role"] + list(dates))

Populate availability table with form responses.

In [4]:
# Iterate over the responses
for i, row in df.iterrows():
    
    # Get the name and role
    name = f"{row['first']} {row['last']}"
    role = row["role"]
    
    # Get the shifts
    shifts = row["shifts"].split(", ")
    year = 2024
    shifts = [pd.to_datetime(f"{shift} {year}").strftime("%-m/%-d/%Y") for shift in shifts]
    
    # Create a row
    schedule = [name, role] + ["" if date in shifts else "x" for date in dates]
    
    # Add the row to the dataframe
    availability.loc[i] = schedule

Process and save availability table.

In [5]:
# Order by role
role_order = ["Senior Clinician", "Junior Clinician", "Senior Director", "Administrative Manager"]
availability["Role"] = pd.Categorical(availability["Role"], categories=role_order, ordered=True)
availability = availability.sort_values(["Role", "Name"]).reset_index(drop=True)

Schedule clinic shifts based on availability and even distribution of shifts.

In [6]:
# First, schedule everyone
schedule = availability.copy()
schedule = schedule.replace("", "s")

# Maximum shifts by role
max_shifts = {
    "Senior Clinician": 6,
    "Junior Clinician": 6,
    "Senior Director": 3,
    "Administrative Manager": 1
} 

# Create counter of shifts per person
schedule["Counts"] = schedule[dates].apply(lambda x: x.value_counts()["s"], axis=1)

Generate schedule table.

In [7]:
# Iterate over the dates
for date in dates:

    # Iterate over the roles
    for role in max_shifts.keys():

        # Get the people who are scheduled
        scheduled = schedule[schedule[date] == "s"]
        scheduled = scheduled[scheduled["Role"] == role]

        # If the number of people is more than the maximum
        if len(scheduled) > max_shifts[role]:
            scheduled = scheduled.sort_values("Counts", ascending=False)

            # Drop people with the fewest shifts until the number is correct
            while len(scheduled) > max_shifts[role]:
                person = scheduled.iloc[0]
                schedule.loc[schedule["Name"] == person["Name"], date] = ""
                schedule.loc[schedule["Name"] == person["Name"], "Counts"] -= 1
                scheduled = scheduled.drop(person.name)

Save to file.

In [8]:
# Save availability and schedule
availability.to_csv("availability.csv", index=False)
schedule.to_csv("schedule.csv", index=False)